In [ ]:
import random
import time
from paho.mqtt import client as mqtt_client

import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


broker = 'test.mosquitto.org'
port = 1883
topic = "Room Temperature"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 1000)}'

ti = []
T = []

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    #client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

# This function is called periodically from FuncAnimation
def animate(i):
    #time.sleep(1)
    temp = random.randint(220, 350)
    temp = temp/10
    T.append(temp)
    ti.append(dt.datetime.now().strftime('%H:%M:%S.%f'))
    msg = f"Temperature: {temp}"
    result = client.publish(topic, msg) # result: [0, 1]
    status = result[0]
    
    if status == 0:
        print(f"Send `{msg}` to topic `{topic}`")
    else:
        print(f"Failed to send message to topic {topic}")

    # Create figure for plotting
    plt.cla()
    plt.plot(ti, T)
    plt.xlabel("Time")
    plt.ylabel("Room Temperature (C)")
    plt.tight_layout()

    print(temp)

def publish(client):
    # Set up plot to call animate() function periodically
    while True:
        ani = FuncAnimation(plt.gcf(), animate, interval=100)
        plt.tight_layout()
        plt.show()

def run():
    client = connect_mqtt()
    client.loop_start()
    publish(client)

if __name__ == '__main__':
    run()